In [1]:
var openDigger = require('../src/open_digger');
const fs = require('fs');

// 时间范围：2024年7月至2025年6月
const startYear = 2024;
const startMonth = 7;
const endYear = 2025;
const endMonth = 6;

// 生成月份标签数组（严格格式："24-07"，仅包含年份后两位和月份，无日期）
const months = [];
for (let year = startYear; year <= endYear; year++) {
    const start = (year === startYear) ? startMonth : 1;
    const end = (year === endYear) ? endMonth : 12;
    for (let month = start; month <= end; month++) {
        // 提取年份后两位（如2024 -> 24）
        const yearStr = year.toString().slice(-2);
        // 月份补零（如7 -> 07）
        const monthStr = month.toString().padStart(2, '0');
        // 组合成"24-07"格式
        months.push(`${yearStr}-${monthStr}`);
    }
}

// 获取平台数据和WHERE条件
var chineseLabel = openDigger.label.getPlatformData([':regions/CN']);
var chineseWhereClause = chineseLabel.map(p => 
    `((repo_id IN (${p.repos.map(r => r.id).join(',')}) AND platform='${p.name}') OR 
    (org_id IN (${p.orgs.map(o => o.id).join(',')}) AND platform='${p.name}'))`
).join(' OR ');

var limit = 15; // 每月返回前15名公司

async function saveMonthlyRankToCSV() {
    const companyData = {};
    const allCompanies = new Set();

    // 循环处理每个月
    for (let year = startYear; year <= endYear; year++) {
        const start = (year === startYear) ? startMonth : 1;
        const end = (year === endYear) ? endMonth : 12;
        
        for (let month = start; month <= end; month++) {
            // 生成当前月份标签（"24-07"格式）
            const yearStr = year.toString().slice(-2);
            const monthStr = month.toString().padStart(2, '0');
            const dateStr = `${yearStr}-${monthStr}`;
            
            console.log(`📅 正在处理 ${dateStr} 数据...`);

            // 查询当月数据
            let data = await openDigger.index.openrank.getRepoOpenrank({
                whereClause: chineseWhereClause,
                startYear: year,
                startMonth: month,
                endYear: year,
                endMonth: month,
                groupBy: 'Company',
                groupTimeRange: 'month',
                order: 'DESC',
                limit: limit
            });

            // 输出表格到控制台
            console.table(data.map((d, i) => ({
                '排名': i + 1,
                '公司': d.name,
                '影响力': d.openrank[0]
            })));

            // 保存当月排名
            data.forEach((d, index) => {
                const rank = index + 1;
                allCompanies.add(d.name);
                if (!companyData[d.name]) {
                    companyData[d.name] = {};
                }
                companyData[d.name][dateStr] = rank;
            });
        }
    }

    // 准备CSV内容，表头为"公司名称" + "24-07"格式的月份
    let csvContent = ['公司名称', ...months].join(',') + '\n';

    // 添加各公司数据
    allCompanies.forEach(company => {
        const row = [company];
        months.forEach(month => {
            row.push(companyData[company][month] || '');
        });
        csvContent += row.join(',') + '\n';
    });

    // 保存为CSV文件
    const fileName = `中国企业月度排名数据_${startYear}${startMonth}_${endYear}${endMonth}.csv`;
    fs.writeFileSync(fileName, csvContent, 'utf8');
    console.log(`✅ 数据已成功保存到 ${fileName}`);
}

// 执行函数
saveMonthlyRankToCSV().catch(err => {
    console.error('❌ 处理过程中发生错误:', err);
});
    

📅 正在处理 24-07 数据...


Promise { <pending> }

┌─────────┬──────┬─────────────┬─────────┐
│ (index) │ 排名 │ 公司        │ 影响力  │
├─────────┼──────┼─────────────┼─────────┤
│ 0       │ 1    │ 'Huawei'    │ 9193.53 │
│ 1       │ 2    │ 'Alibaba'   │ 2106.11 │
│ 2       │ 3    │ 'Ant group' │ 1537.58 │
│ 3       │ 4    │ 'Baidu'     │ 1317.53 │
│ 4       │ 5    │ 'DaoCloud'  │ 1184.6  │
│ 5       │ 6    │ 'Others'    │ 1047.88 │
│ 6       │ 7    │ 'ByteDance' │ 703.74  │
│ 7       │ 8    │ 'Tencent'   │ 533.19  │
│ 8       │ 9    │ 'ESPRESSIF' │ 531.99  │
│ 9       │ 10   │ 'Fit2Cloud' │ 515.32  │
│ 10      │ 11   │ 'PingCAP'   │ 419.05  │
│ 11      │ 12   │ 'openKylin' │ 346.19  │
│ 12      │ 13   │ 'Zilliz'    │ 317.98  │
│ 13      │ 14   │ 'StarRocks' │ 259.9   │
│ 14      │ 15   │ 'Deepin'    │ 198.58  │
└─────────┴──────┴─────────────┴─────────┘
📅 正在处理 24-08 数据...
┌─────────┬──────┬─────────────┬─────────┐
│ (index) │ 排名 │ 公司        │ 影响力  │
├─────────┼──────┼─────────────┼─────────┤
│ 0       │ 1    │ 'Huawei'    │ 9859.37 │
│ 1   